## Instructions ##
971 yeast

linear regression, snp weight vector, 选top 10%， 20%， 30%。。

分别用这些snp来再跑linear regression的model( no gene)

再把这些snp放到gene上，再跑一次linear regression model

 

神经网络input snp

最多两千个feature，用linear regression来筛选

 

还要考虑promoter， enhancer（先不搞）

 

1. 先选snp （Science paper：看snp preproccesing的procedure）<5 删掉

2. fit linear regression model

3. 选大约2000个

4. 再用神经网络 （pytorch）



In [2]:
import pandas as pd 
import numpy as np
import bisect as bs
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import pandas as pd
import scipy.stats as stat


In [3]:
SNP_url = "./1011geneSNP.csv"
Gene_url = "./GCF_000146045.2_R64_genomic.gff"

snp_yeast_matrix = pd.read_csv(SNP_url, sep = ',' , error_bad_lines=False)
del snp_yeast_matrix['#CHROM']
del snp_yeast_matrix['POS']
del snp_yeast_matrix['REF']
del snp_yeast_matrix['ALT']
del snp_yeast_matrix['ANN[*].GENE']
del snp_yeast_matrix['ANN[*].GENEID']
index_list = snp_yeast_matrix["ID"].tolist()
del snp_yeast_matrix['ID']
snp_yeast_matrix = snp_yeast_matrix.transpose()
column_names = list(snp_yeast_matrix.index)
to_delete = ["ABC", "ABF", "ASP", "BGS","BGF", "BHL","BHQ","BID","BIR","BKG","BKN","CFH","SACE_YAL","SACE_YBA",
             "SACE_YBB","SACE_YBM","SACE_YBN","SACE_YBO","SACE_YBP","SACE_YCJ","SACE_YCS","SACE_YCT","SACE_YCU","SACE_YCV","SACE_YCW",
             "SACE_YCX","SACE_YCY","SACE_YCZ","SACE_YDA","SACE_YDB","SACE_YDC","SACE_YDD", "SACE_YDE","SACE_YDF","SACE_YDG","SACE_YDH","SACE_YDI",
             "SACE_YDJ", "SACE_YDK", "SACE_YDL"]
print(len(to_delete))
index = 0
for index in range(len(to_delete)):
    curr = to_delete[index]
    curr = curr+"_"+curr
    to_delete[index] = curr
    index+=1
to_delete_index = []
for item in to_delete:
    curr_index = column_names.index(item)
    to_delete_index.append(curr_index)
snp_yeast_matrix = snp_yeast_matrix.drop(to_delete)
snp_yeast_matrix.head()

40


,0,1,2,3,4,5,6,7,8,9,...,18921,18922,18923,18924,18925,18926,18927,18928,18929,18930
AAA_AAA,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
AAB_AAB,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
AAC_AAC,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAD_AAD,0,0,0,0,1,1,1,0,0,0,...,0,0,1,1,1,1,0,0,1,1
AAE_AAE,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
pheno_url = "./phenoMatrix_35ConditionsNormalizedByYPD.csv"
pheno_yeast_matrix = pd.read_csv(pheno_url, delimiter='\t', error_bad_lines=False)
YPD6AU = pheno_yeast_matrix["YPD6AU"]
YPDBENOMYL200 = pheno_yeast_matrix["YPDBENOMYL200"]
YPDNACL1M = pheno_yeast_matrix["YPDNACL1M"]
YPDSDS = pheno_yeast_matrix["YPDSDS"]
YPGALACTOSE = pheno_yeast_matrix["YPGALACTOSE"]
YPGLYCEROL = pheno_yeast_matrix["YPGLYCEROL"]
YPRIBOSE = pheno_yeast_matrix["YPRIBOSE"]
YPSORBITOL = pheno_yeast_matrix["YPSORBITOL"]
total_run = 100

In [5]:
sum = 0
weight_vector_1 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPD6AU, test_size=0.2)
    lr1 = linear_model.LinearRegression()
    lr1.fit(x_train, y_train)
    temp = np.array(lr1.coef_)
    weight_vector_1 = [x + y for x, y in zip(weight_vector_1, temp)]
    predictions_1 = lr1.predict(x_test)
    pcc_1 = stat.pearsonr(predictions_1, y_test)
    counter+=1
    sum += pcc_1[0]

avg_assoc_1 = sum/total_run
avg_weight_vector_1 = np.divide(weight_vector_1, total_run)
print(avg_assoc_1)
print(avg_weight_vector_1)
top2000_index = (-avg_weight_vector_1).argsort()[:2000]
print (top2000_index)
print (avg_weight_vector_1[top2000_index])

0.017625507644391997
[-2.40193329e+08 -9.51329668e+07 -6.91509560e+07 ... -2.07819641e+05
  1.72123812e+06 -3.65489209e+06]
[   22    40   177 ... 12580 13439  1064]
[35909712.86755651 30969496.06911329 30172606.27411069 ...
  2057249.79204882  2056066.05203741  2055314.69394521]


In [6]:
sum = 0
weight_vector_3 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDNACL1M, test_size=0.2)
    lr3 = linear_model.LinearRegression()
    lr3.fit(x_train, y_train)
    temp = np.array(lr3.coef_)
    weight_vector_3 = [x + y for x, y in zip(weight_vector_3, temp)]
    predictions_3 = lr3.predict(x_test)
    pcc_3 = stat.pearsonr(predictions_3, y_test)
    counter+=1
    sum += pcc_3[0]

avg_assoc_3 = sum/total_run
avg_weight_vector_3 = np.divide(weight_vector_3, total_run)
print(avg_assoc_3)
print(avg_weight_vector_3)
top2000_index_3 = (-avg_weight_vector_3).argsort()[:2000]
print (top2000_index_3)
print (avg_weight_vector_3[top2000_index_3])

0.037436911180733945
[-3.37849673e+05  6.46576234e+07 -1.76284387e+07 ...  5.74838207e+04
  6.64331609e+05 -1.09117706e+06]
[    1     4     7 ... 17459  8807   579]
[64657623.42498443 43433330.5155755  14566282.72546469 ...
   724000.37730427   723879.2780653    723841.16101986]


In [7]:
sum = 0
weight_vector_4 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDSDS, test_size=0.2)
    lr4 = linear_model.LinearRegression()
    lr4.fit(x_train, y_train)
    temp = np.array(lr4.coef_)
    weight_vector_4 = [x + y for x, y in zip(weight_vector_4, temp)]
    predictions_4 = lr4.predict(x_test)
    pcc_4 = stat.pearsonr(predictions_4, y_test)
    counter+=1
    sum += pcc_4[0]

avg_assoc_4 = sum/total_run
avg_weight_vector_4 = np.divide(weight_vector_4, total_run)
print(avg_assoc_4)
print(avg_weight_vector_4)
top2000_index_4 = (-avg_weight_vector_4).argsort()[:2000]
print (top2000_index_4)
print (avg_weight_vector_4[top2000_index_4])

0.050776698519380235
[ 1.80293179e+08  3.00320564e+07  1.00394208e+07 ... -3.23214271e+04
 -1.37441458e+06 -3.48155840e+05]
[   0    9    1 ... 8013 2623 8329]
[1.80293179e+08 3.77394678e+07 3.00320564e+07 ... 7.83034738e+05
 7.82003570e+05 7.81750319e+05]


In [8]:
sum = 0
weight_vector_2 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDBENOMYL200, test_size=0.2)
    lr2 = linear_model.LinearRegression()
    lr2.fit(x_train, y_train)
    temp = np.array(lr2.coef_)
    weight_vector_2 = [x + y for x, y in zip(weight_vector_2, temp)]
    predictions_2 = lr2.predict(x_test)
    pcc_2 = stat.pearsonr(predictions_2, y_test)
    counter+=1
    sum += pcc_2[0]

avg_assoc_2 = sum/total_run
avg_weight_vector_2 = np.divide(weight_vector_2, total_run)
print(avg_assoc_2)
print(avg_weight_vector_2)
top2000_index_2 = (-avg_weight_vector_2).argsort()[:2000]
print (top2000_index_2)
print (avg_weight_vector_2[top2000_index_2])

0.01487416324101254
[ -7086718.755928    19818053.69945217 -10674090.39858287 ...
    368086.6244495    -112899.05171859   -325806.63563745]
[    4     6     1 ... 15876 13231  7014]
[28971723.24723437 20022340.91732469 19818053.69945217 ...
   698925.65741911   698919.24679987   698725.96386096]


In [9]:
sum = 0
weight_vector_5 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPGALACTOSE, test_size=0.2)
    lr5 = linear_model.LinearRegression()
    lr5.fit(x_train, y_train)
    temp = np.array(lr5.coef_)
    weight_vector_5 = [x + y for x, y in zip(weight_vector_5, temp)]
    predictions_5 = lr5.predict(x_test)
    pcc_5 = stat.pearsonr(predictions_5, y_test)
    counter+=1
    sum += pcc_5[0]

avg_assoc_5 = sum/total_run
avg_weight_vector_5 = np.divide(weight_vector_5, total_run)
print(avg_assoc_5)
print(avg_weight_vector_5)
top2000_index_5 = (-avg_weight_vector_5).argsort()[:2000]
print (top2000_index_5)
print (avg_weight_vector_5[top2000_index_5])

0.027988724790353886
[-3.55220592e+08  1.09278129e+08 -2.61123680e+08 ... -4.55756319e+05
  2.54604957e+06 -5.72904128e+04]
[    1    11    10 ...  3537 12134 11862]
[1.09278129e+08 7.41107208e+07 6.94411198e+07 ... 1.69495390e+06
 1.69492387e+06 1.69119076e+06]


In [10]:
sum = 0
weight_vector_6 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPGLYCEROL, test_size=0.2)
    lr6 = linear_model.LinearRegression()
    lr6.fit(x_train, y_train)
    temp = np.array(lr6.coef_)
    weight_vector_6 = [x + y for x, y in zip(weight_vector_6, temp)]
    predictions_6 = lr6.predict(x_test)
    pcc_6 = stat.pearsonr(predictions_6, y_test)
    counter+=1
    sum += pcc_6[0]

avg_assoc_6 = sum/total_run
avg_weight_vector_6 = np.divide(weight_vector_6, total_run)
print(avg_assoc_6)
print(avg_weight_vector_6)
top2000_index_6 = (-avg_weight_vector_6).argsort()[:2000]
print (top2000_index_6)
print (avg_weight_vector_6[top2000_index_6])

0.01669042277109073
[ 9.57655442e+07  8.30312598e+07 -5.45238087e+07 ... -6.39761225e+03
 -4.70708416e+06  2.03677672e+06]
[    0     1     4 ... 13165 16181  7643]
[95765544.23048124 83031259.82149577 63159836.09703083 ...
  4184377.91238532  4183117.47498691  4180022.78792203]


In [11]:
sum = 0
weight_vector_7 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPRIBOSE, test_size=0.2)
    lr7 = linear_model.LinearRegression()
    lr7.fit(x_train, y_train)
    temp = np.array(lr7.coef_)
    weight_vector_7 = [x + y for x, y in zip(weight_vector_7, temp)]
    predictions_7 = lr7.predict(x_test)
    pcc_7 = stat.pearsonr(predictions_7, y_test)
    counter+=1
    sum += pcc_7[0]

avg_assoc_7 = sum/total_run
avg_weight_vector_7 = np.divide(weight_vector_7, total_run)
print(avg_assoc_7)
print(avg_weight_vector_7)
top2000_index_7 = (-avg_weight_vector_7).argsort()[:2000]
print (top2000_index_7)
print (avg_weight_vector_7[top2000_index_7])

0.012708775898420057
[ 1.37363610e+08  2.55347170e+07 -2.27010896e+07 ... -5.95999093e+05
 -1.03600312e+04  1.72148497e+06]
[    0     3    22 ...  9194  6502 15553]
[1.37363610e+08 3.90485854e+07 3.55737953e+07 ... 1.04004830e+06
 1.03954627e+06 1.03873070e+06]


In [12]:
sum = 0
weight_vector_8 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPSORBITOL, test_size=0.2)
    lr8 = linear_model.LinearRegression()
    lr8.fit(x_train, y_train)
    temp = np.array(lr8.coef_)
    weight_vector_8 = [x + y for x, y in zip(weight_vector_8, temp)]
    predictions_8 = lr8.predict(x_test)
    pcc_8 = stat.pearsonr(predictions_8, y_test)
    counter+=1
    sum += pcc_8[0]

avg_assoc_8 = sum/total_run
avg_weight_vector_8 = np.divide(weight_vector_8, total_run)
print(avg_assoc_8)
print(avg_weight_vector_8)
top2000_index_8 = (-avg_weight_vector_8).argsort()[:2000]
print (top2000_index_8)
print (avg_weight_vector_8[top2000_index_8])

0.00658958047780311
[-1.98248928e+08  2.49733290e+07 -2.84093445e+07 ...  6.25713982e+05
  3.72645493e+05 -1.22592640e+06]
[  14   18   19 ... 3716 2050 3460]
[56802836.23059125 42934110.51236925 37816279.76193699 ...
  2528406.61915241  2527849.51669522  2527272.76368826]


In [13]:
df_columns = ["YPD6AU", "YPDBENOMYL200", "YPDNACL1M", "YPDSDS", "YPGALACTOSE", "YPGLYCEROL", "YPRIBOSE", "YPSORBITOL"]
index_df = pd.DataFrame(list(zip(top2000_index, top2000_index_2, top2000_index_3, top2000_index_4, top2000_index_5, top2000_index_6, top2000_index_7, top2000_index_8)), columns = df_columns)


最后一层hiddeen nueron 数量为一

2000 1000 500 100 1 

adam optimize 要

batch normalization 

.cuda 挪到gpu

batch training 

batch

# Neural Network #

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as dt

class Net(nn.Module):
    def __init__(self, input_size, h1, h2, h3, output_size, lr):
        super(Net, self).__init__()
        self.l1 = nn.Linear(input_size, h1)
        self.l2 = nn.Linear(h1, h2)
        self.l3 = nn.Linear(h2, h3)
        self.l4 = nn.Linear(h3, output_size)
        self.relu = nn.ReLU()
        
        self.loss_fn = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr)

    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        out = self.relu(out)
        out = self.l4(out)
        return out
    
    #Not used. 
    def train_net(self, iterations, train_data, target):
        for epoch in range (iterations):
            net.optimizer.zero_grad()
            output = net.forward(train_data)
            loss = net.loss_fn(output, target)
            #print(list(self.parameters()))
            print("current loss is: %f"%loss)
            loss.backward()
            net.optimizer.step()
            epoch += 1

In [26]:
device = torch.device('cuda')
input_size = 2000
h1 = 1000
h2 = 500
h3 = 100
output_size = 1
lr = 0.001
num_epochs = 100
batch_size = 200

traning_data = torch.tensor(snp_yeast_matrix[top2000_index].values, dtype = torch.float)
answer = torch.tensor(YPD6AU, dtype = torch.float)
train_set = dt.TensorDataset(traning_data, answer)
train_loader = dt.DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)

net = Net(input_size, h1, h2, h3, output_size, lr).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr)

total_step = len(train_loader)


In [27]:
for epoch in range (num_epochs):
    for i, (data, target) in enumerate(train_loader):
        net.zero_grad()
        data = data.to(device)
        target = target.to(device)
        output = net.forward(data).to(device)
        loss = net.loss_fn(output, target)

        #print(list(self.parameters()))
        #print("current loss is: %f"%loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch += 1
        
        if (i) % 100 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}' 
                   .format(epoch, num_epochs, loss.item()))

C:\Users\Charles\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:443: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/100], Loss: 0.0443


C:\Users\Charles\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:443: UserWarning: Using a target size (torch.Size([171])) that is different to the input size (torch.Size([171, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [2/100], Loss: 0.0301
Epoch [3/100], Loss: 0.0288
Epoch [4/100], Loss: 0.0205
Epoch [5/100], Loss: 0.0175
Epoch [6/100], Loss: 0.0163
Epoch [7/100], Loss: 0.0182
Epoch [8/100], Loss: 0.0188
Epoch [9/100], Loss: 0.0208
Epoch [10/100], Loss: 0.0182
Epoch [11/100], Loss: 0.0184
Epoch [12/100], Loss: 0.0201
Epoch [13/100], Loss: 0.0187
Epoch [14/100], Loss: 0.0164
Epoch [15/100], Loss: 0.0195
Epoch [16/100], Loss: 0.0190
Epoch [17/100], Loss: 0.0220
Epoch [18/100], Loss: 0.0167
Epoch [19/100], Loss: 0.0197
Epoch [20/100], Loss: 0.0224
Epoch [21/100], Loss: 0.0200
Epoch [22/100], Loss: 0.0186
Epoch [23/100], Loss: 0.0201
Epoch [24/100], Loss: 0.0152
Epoch [25/100], Loss: 0.0235
Epoch [26/100], Loss: 0.0186
Epoch [27/100], Loss: 0.0182
Epoch [28/100], Loss: 0.0175
Epoch [29/100], Loss: 0.0203
Epoch [30/100], Loss: 0.0155
Epoch [31/100], Loss: 0.0162
Epoch [32/100], Loss: 0.0176
Epoch [33/100], Loss: 0.0193
Epoch [34/100], Loss: 0.0231
Epoch [35/100], Loss: 0.0201
Epoch [36/100], Loss: 

In [69]:
split = 0.8
train_size = int(split * len(train_set))
print(train_size)
test_size = len(train_set) - train_size 
print(test_size)

train_dataset, test_dataset = dt.random_split(train_set, [train_size, test_size])

device = torch.device('cuda')
input_size = 2000
h1 = 1000
h2 = 500
h3 = 100
output_size = 1
lr = 0.00001
num_epochs = 1000
batch_size = 200
train_loader = dt.DataLoader(dataset=train_dataset, batch_size=batch_size)

net = Net(input_size, h1, h2, h3, output_size, lr).to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr)
test_loader = dt.DataLoader(dataset=test_dataset, batch_size=batch_size)

def check_correlation(test_loader, net):
    counter = 0
    with torch.no_grad():
        total_err = 0.0
        for (data, target) in test_loader:
            data = data.to(device)
            target = target.cpu().numpy()
            output = torch.squeeze(net(data)).cpu().numpy()
            result = stat.pearsonr(target, output)
    print("Correlation: %f \n P-Value: %f" %(result[0], result[1]))

776
195


In [70]:
for epoch in range (num_epochs):
    total_loss = 0
    counter = 0
    for i, (data, target) in enumerate(train_loader):
        net.zero_grad()
        data = data.to(device)
        target = target.to(device)
        output = net.forward(data).to(device)
        loss = net.loss_fn(output, target)
        total_loss += loss
        counter += 1
        #print(list(self.parameters()))
        #print("current loss is: %f"%loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    check_correlation(test_loader, net)    
    epoch += 1

    print ('Epoch [{}/{}], Loss: {:.4f}' 
            .format(epoch, num_epochs, total_loss/len(train_loader)))

Correlation: -0.242963 
 P-Value: 0.000621
Epoch [1/1000], Loss: 0.1715
Correlation: -0.219390 
 P-Value: 0.002059
Epoch [2/1000], Loss: 0.1486
Correlation: -0.210183 
 P-Value: 0.003186
Epoch [3/1000], Loss: 0.1293
Correlation: -0.207262 
 P-Value: 0.003646
Epoch [4/1000], Loss: 0.1132
Correlation: -0.202118 
 P-Value: 0.004603
Epoch [5/1000], Loss: 0.0995
Correlation: -0.198553 
 P-Value: 0.005393
Epoch [6/1000], Loss: 0.0876
Correlation: -0.196516 
 P-Value: 0.005897
Epoch [7/1000], Loss: 0.0779
Correlation: -0.195444 
 P-Value: 0.006179
Epoch [8/1000], Loss: 0.0704
Correlation: -0.194631 
 P-Value: 0.006400
Epoch [9/1000], Loss: 0.0648
Correlation: -0.193730 
 P-Value: 0.006655
Epoch [10/1000], Loss: 0.0602
Correlation: -0.191898 
 P-Value: 0.007199
Epoch [11/1000], Loss: 0.0557
Correlation: -0.188963 
 P-Value: 0.008154
Epoch [12/1000], Loss: 0.0512
Correlation: -0.184145 
 P-Value: 0.009966
Epoch [13/1000], Loss: 0.0468
Correlation: -0.176340 
 P-Value: 0.013664
Epoch [14/1000], 

 P-Value: 0.330007
Epoch [113/1000], Loss: 0.0187
Correlation: -0.070154 
 P-Value: 0.329781
Epoch [114/1000], Loss: 0.0187
Correlation: -0.070141 
 P-Value: 0.329870
Epoch [115/1000], Loss: 0.0187
Correlation: -0.070136 
 P-Value: 0.329905
Epoch [116/1000], Loss: 0.0187
Correlation: -0.070148 
 P-Value: 0.329825
Epoch [117/1000], Loss: 0.0187
Correlation: -0.070132 
 P-Value: 0.329936
Epoch [118/1000], Loss: 0.0187
Correlation: -0.070097 
 P-Value: 0.330180
Epoch [119/1000], Loss: 0.0187
Correlation: -0.070054 
 P-Value: 0.330478
Epoch [120/1000], Loss: 0.0186
Correlation: -0.070017 
 P-Value: 0.330735
Epoch [121/1000], Loss: 0.0186
Correlation: -0.069999 
 P-Value: 0.330853
Epoch [122/1000], Loss: 0.0186
Correlation: -0.070030 
 P-Value: 0.330638
Epoch [123/1000], Loss: 0.0186
Correlation: -0.070064 
 P-Value: 0.330408
Epoch [124/1000], Loss: 0.0186
Correlation: -0.070085 
 P-Value: 0.330260
Epoch [125/1000], Loss: 0.0186
Correlation: -0.070102 
 P-Value: 0.330145
Epoch [126/1000], L

 P-Value: 0.389918
Epoch [224/1000], Loss: 0.0184
Correlation: -0.061769 
 P-Value: 0.390989
Epoch [225/1000], Loss: 0.0184
Correlation: -0.061645 
 P-Value: 0.391942
Epoch [226/1000], Loss: 0.0184
Correlation: -0.061492 
 P-Value: 0.393121
Epoch [227/1000], Loss: 0.0183
Correlation: -0.061343 
 P-Value: 0.394265
Epoch [228/1000], Loss: 0.0183
Correlation: -0.061220 
 P-Value: 0.395219
Epoch [229/1000], Loss: 0.0183
Correlation: -0.061117 
 P-Value: 0.396013
Epoch [230/1000], Loss: 0.0183
Correlation: -0.060966 
 P-Value: 0.397186
Epoch [231/1000], Loss: 0.0183
Correlation: -0.060848 
 P-Value: 0.398098
Epoch [232/1000], Loss: 0.0183
Correlation: -0.060739 
 P-Value: 0.398947
Epoch [233/1000], Loss: 0.0183
Correlation: -0.060610 
 P-Value: 0.399950
Epoch [234/1000], Loss: 0.0183
Correlation: -0.060465 
 P-Value: 0.401085
Epoch [235/1000], Loss: 0.0183
Correlation: -0.060356 
 P-Value: 0.401935
Epoch [236/1000], Loss: 0.0183
Correlation: -0.060254 
 P-Value: 0.402731
Epoch [237/1000], L

 P-Value: 0.477110
Epoch [335/1000], Loss: 0.0183
Correlation: -0.051204 
 P-Value: 0.477147
Epoch [336/1000], Loss: 0.0183
Correlation: -0.051174 
 P-Value: 0.477412
Epoch [337/1000], Loss: 0.0183
Correlation: -0.051083 
 P-Value: 0.478196
Epoch [338/1000], Loss: 0.0183
Correlation: -0.051030 
 P-Value: 0.478650
Epoch [339/1000], Loss: 0.0183
Correlation: -0.051030 
 P-Value: 0.478646
Epoch [340/1000], Loss: 0.0183
Correlation: -0.050944 
 P-Value: 0.479392
Epoch [341/1000], Loss: 0.0183
Correlation: -0.050901 
 P-Value: 0.479768
Epoch [342/1000], Loss: 0.0183
Correlation: -0.050857 
 P-Value: 0.480141
Epoch [343/1000], Loss: 0.0183
Correlation: -0.050816 
 P-Value: 0.480496
Epoch [344/1000], Loss: 0.0183
Correlation: -0.050765 
 P-Value: 0.480937
Epoch [345/1000], Loss: 0.0183
Correlation: -0.050727 
 P-Value: 0.481265
Epoch [346/1000], Loss: 0.0183
Correlation: -0.050652 
 P-Value: 0.481921
Epoch [347/1000], Loss: 0.0183
Correlation: -0.050633 
 P-Value: 0.482087
Epoch [348/1000], L

 P-Value: 0.511132
Epoch [446/1000], Loss: 0.0183
Correlation: -0.047243 
 P-Value: 0.511926
Epoch [447/1000], Loss: 0.0183
Correlation: -0.047258 
 P-Value: 0.511793
Epoch [448/1000], Loss: 0.0183
Correlation: -0.047235 
 P-Value: 0.512001
Epoch [449/1000], Loss: 0.0183
Correlation: -0.047188 
 P-Value: 0.512425
Epoch [450/1000], Loss: 0.0183
Correlation: -0.047162 
 P-Value: 0.512657
Epoch [451/1000], Loss: 0.0183
Correlation: -0.047110 
 P-Value: 0.513116
Epoch [452/1000], Loss: 0.0183
Correlation: -0.047101 
 P-Value: 0.513197
Epoch [453/1000], Loss: 0.0183
Correlation: -0.047068 
 P-Value: 0.513492
Epoch [454/1000], Loss: 0.0183
Correlation: -0.046970 
 P-Value: 0.514374
Epoch [455/1000], Loss: 0.0183
Correlation: -0.046991 
 P-Value: 0.514181
Epoch [456/1000], Loss: 0.0183
Correlation: -0.047041 
 P-Value: 0.513736
Epoch [457/1000], Loss: 0.0183
Correlation: -0.046869 
 P-Value: 0.515278
Epoch [458/1000], Loss: 0.0183
Correlation: -0.046815 
 P-Value: 0.515764
Epoch [459/1000], L

 P-Value: 0.547229
Epoch [557/1000], Loss: 0.0183
Correlation: -0.043334 
 P-Value: 0.547493
Epoch [558/1000], Loss: 0.0183
Correlation: -0.043244 
 P-Value: 0.548328
Epoch [559/1000], Loss: 0.0183
Correlation: -0.043217 
 P-Value: 0.548577
Epoch [560/1000], Loss: 0.0183
Correlation: -0.043164 
 P-Value: 0.549066
Epoch [561/1000], Loss: 0.0183
Correlation: -0.043160 
 P-Value: 0.549106
Epoch [562/1000], Loss: 0.0183
Correlation: -0.043169 
 P-Value: 0.549022
Epoch [563/1000], Loss: 0.0183
Correlation: -0.043172 
 P-Value: 0.548991
Epoch [564/1000], Loss: 0.0183
Correlation: -0.043181 
 P-Value: 0.548912
Epoch [565/1000], Loss: 0.0183
Correlation: -0.043121 
 P-Value: 0.549469
Epoch [566/1000], Loss: 0.0183
Correlation: -0.043104 
 P-Value: 0.549628
Epoch [567/1000], Loss: 0.0183
Correlation: -0.043143 
 P-Value: 0.549263
Epoch [568/1000], Loss: 0.0183
Correlation: -0.043113 
 P-Value: 0.549537
Epoch [569/1000], Loss: 0.0183
Correlation: -0.043115 
 P-Value: 0.549519
Epoch [570/1000], L

 P-Value: 0.567164
Epoch [668/1000], Loss: 0.0183
Correlation: -0.041188 
 P-Value: 0.567522
Epoch [669/1000], Loss: 0.0183
Correlation: -0.041013 
 P-Value: 0.569176
Epoch [670/1000], Loss: 0.0183
Correlation: -0.041088 
 P-Value: 0.568460
Epoch [671/1000], Loss: 0.0183
Correlation: -0.041087 
 P-Value: 0.568471
Epoch [672/1000], Loss: 0.0183
Correlation: -0.041006 
 P-Value: 0.569235
Epoch [673/1000], Loss: 0.0183
Correlation: -0.041116 
 P-Value: 0.568199
Epoch [674/1000], Loss: 0.0183
Correlation: -0.041074 
 P-Value: 0.568595
Epoch [675/1000], Loss: 0.0183
Correlation: -0.040949 
 P-Value: 0.569771
Epoch [676/1000], Loss: 0.0183
Correlation: -0.041048 
 P-Value: 0.568844
Epoch [677/1000], Loss: 0.0183
Correlation: -0.040969 
 P-Value: 0.569586
Epoch [678/1000], Loss: 0.0183
Correlation: -0.040847 
 P-Value: 0.570740
Epoch [679/1000], Loss: 0.0183
Correlation: -0.041000 
 P-Value: 0.569290
Epoch [680/1000], Loss: 0.0183
Correlation: -0.040965 
 P-Value: 0.569621
Epoch [681/1000], L

 P-Value: 0.589310
Epoch [779/1000], Loss: 0.0183
Correlation: -0.038961 
 P-Value: 0.588666
Epoch [780/1000], Loss: 0.0183
Correlation: -0.038855 
 P-Value: 0.589679
Epoch [781/1000], Loss: 0.0183
Correlation: -0.038806 
 P-Value: 0.590153
Epoch [782/1000], Loss: 0.0183
Correlation: -0.038920 
 P-Value: 0.589063
Epoch [783/1000], Loss: 0.0183
Correlation: -0.038891 
 P-Value: 0.589342
Epoch [784/1000], Loss: 0.0183
Correlation: -0.038724 
 P-Value: 0.590940
Epoch [785/1000], Loss: 0.0183
Correlation: -0.038821 
 P-Value: 0.590012
Epoch [786/1000], Loss: 0.0183
Correlation: -0.038947 
 P-Value: 0.588797
Epoch [787/1000], Loss: 0.0183
Correlation: -0.038747 
 P-Value: 0.590721
Epoch [788/1000], Loss: 0.0183
Correlation: -0.038542 
 P-Value: 0.592682
Epoch [789/1000], Loss: 0.0183
Correlation: -0.038781 
 P-Value: 0.590393
Epoch [790/1000], Loss: 0.0183
Correlation: -0.038788 
 P-Value: 0.590329
Epoch [791/1000], Loss: 0.0183
Correlation: -0.038584 
 P-Value: 0.592286
Epoch [792/1000], L

 P-Value: 0.608362
Epoch [890/1000], Loss: 0.0183
Correlation: -0.036835 
 P-Value: 0.609185
Epoch [891/1000], Loss: 0.0183
Correlation: -0.036885 
 P-Value: 0.608698
Epoch [892/1000], Loss: 0.0183
Correlation: -0.036939 
 P-Value: 0.608175
Epoch [893/1000], Loss: 0.0183
Correlation: -0.036828 
 P-Value: 0.609255
Epoch [894/1000], Loss: 0.0183
Correlation: -0.036620 
 P-Value: 0.611273
Epoch [895/1000], Loss: 0.0183
Correlation: -0.036795 
 P-Value: 0.609573
Epoch [896/1000], Loss: 0.0183
Correlation: -0.036945 
 P-Value: 0.608111
Epoch [897/1000], Loss: 0.0183
Correlation: -0.036818 
 P-Value: 0.609346
Epoch [898/1000], Loss: 0.0183
Correlation: -0.036701 
 P-Value: 0.610489
Epoch [899/1000], Loss: 0.0183
Correlation: -0.036645 
 P-Value: 0.611034
Epoch [900/1000], Loss: 0.0183
Correlation: -0.036695 
 P-Value: 0.610541
Epoch [901/1000], Loss: 0.0183
Correlation: -0.036788 
 P-Value: 0.609644
Epoch [902/1000], Loss: 0.0183
Correlation: -0.036655 
 P-Value: 0.610931
Epoch [903/1000], L